<a href="https://colab.research.google.com/github/sakuronohana/my_datascience/blob/master/projects/airline_delay/Airline_Delay_Pred.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Prediction einer Flugverspätung

In dieser Trainingssession werden wir eine Klassifikation mittels KNN durchführen. Zu diesem Zweck werden wir auf einen auf Kaggle zur Verfügung gestellten Datensatz zurückgreifen. 

[Quelle: Kaggle](https://www.kaggle.com/datasets/jimschacko/airlines-dataset-to-predict-a-delay)

In [78]:
import numpy as np
import pandas as pd
import tensorflow as tf

tf.__version__

'2.8.2'

In [79]:
dataloc = 'https://raw.githubusercontent.com/sakuronohana/my_datascience/master/projects/airline_delay/Airlines.csv'

dataset = pd.read_csv(dataloc)
dataset.head()

,id,Airline,Flight,AirportFrom,AirportTo,DayOfWeek,Time,Length,Delay
0,1,CO,269,SFO,IAH,3,15,205,1
1,2,US,1558,PHX,CLT,3,15,222,1
2,3,AA,2400,LAX,DFW,3,20,165,1
3,4,AA,2466,SFO,DFW,3,20,195,1
4,5,AS,108,ANC,SEA,3,30,202,0


In [ ]:
dataset.isnull().sum()

In [80]:
X = dataset.iloc[:,1:-1]
y = dataset.iloc[:,-1]

In [ ]:
dataset.groupby(by=['Airline'], axis=0).sum()
dataset.groupby(by=['AirportFrom'], axis=0).sum()
dataset.groupby(by=['AirportTo'], axis=0).sum()

In [81]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer

# Selektive Datentransformierung auf Spalte 0, 2 und 3
ohe = OneHotEncoder(categories='auto')
X = ohe.fit_transform(X[['Airline','AirportFrom','AirportTo']]).toarray()

In [84]:
X = np.array()

In [85]:
print(h_labels)

[array(['9E', 'AA', 'AS', 'B6', 'CO', 'DL', 'EV', 'F9', 'FL', 'HA', 'MQ',
       'OH', 'OO', 'UA', 'US', 'WN', 'XE', 'YV'], dtype=object), array(['ABE', 'ABI', 'ABQ', 'ABR', 'ABY', 'ACT', 'ACV', 'ACY', 'ADK',
       'ADQ', 'AEX', 'AGS', 'ALB', 'AMA', 'ANC', 'ASE', 'ATL', 'ATW',
       'AUS', 'AVL', 'AVP', 'AZO', 'BDL', 'BET', 'BFL', 'BGM', 'BGR',
       'BHM', 'BIL', 'BIS', 'BKG', 'BLI', 'BMI', 'BNA', 'BOI', 'BOS',
       'BQK', 'BQN', 'BRO', 'BRW', 'BTM', 'BTR', 'BTV', 'BUF', 'BUR',
       'BWI', 'BZN', 'CAE', 'CAK', 'CDC', 'CDV', 'CEC', 'CHA', 'CHO',
       'CHS', 'CIC', 'CID', 'CLD', 'CLE', 'CLL', 'CLT', 'CMH', 'CMI',
       'CMX', 'COD', 'COS', 'COU', 'CPR', 'CRP', 'CRW', 'CSG', 'CVG',
       'CWA', 'CYS', 'DAB', 'DAL', 'DAY', 'DBQ', 'DCA', 'DEN', 'DFW',
       'DHN', 'DLH', 'DRO', 'DSM', 'DTW', 'EAU', 'ECP', 'EGE', 'EKO',
       'ELM', 'ELP', 'ERI', 'EUG', 'EVV', 'EWN', 'EWR', 'EYW', 'FAI',
       'FAR', 'FAT', 'FAY', 'FCA', 'FLG', 'FLL', 'FLO', 'FNT', 'FSD',
       'FSM', 'FWA', 

In [52]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2, random_state = 0)

TypeError: ignored